In [1]:
from transformers import AutoModel, AutoTokenizer
import torch
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity

In [3]:
!pip install InstructorEmbedding

In [5]:
!pip install sentence-transformers==2.2.2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 3.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached 

In [6]:
from InstructorEmbedding import INSTRUCTOR

model = INSTRUCTOR('hkunlp/instructor-large')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


.gitattributes:   0%|          | 0.00/1.48k [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/270 [00:00<?, ?B/s]

2_Dense/config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/3.15M [00:00<?, ?B/s]

...

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.41k [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/461 [00:00<?, ?B/s]

load INSTRUCTOR_Transformer
max_seq_length  512


In [75]:
df = pd.read_csv('/content/cv-valid-dev.csv')

df.head(5)

,filename,text,up_votes,down_votes,age,gender,accent,duration,generated_text,finetuned_text
0,cv-valid-dev/sample-000000.mp3,be careful with your prognostications said the...,1,0,NaN,NaN,NaN,NaN,BE CAREFUL WITH YOUR PROGNOSTICATIONS SAID THE...,BE CAREFUL WITH YOUR PROCNOSTICATIONS SAID THE...
1,cv-valid-dev/sample-000001.mp3,then why should they be surprised when they se...,2,0,NaN,NaN,NaN,NaN,THEN WHY SHOULD THEY BE SURPRISED WHEN THEY SE...,THEN WHY SHOULD THEY BE SURPRISED WITH THE SEAONE
2,cv-valid-dev/sample-000002.mp3,a young arab also loaded down with baggage ent...,2,0,NaN,NaN,NaN,NaN,A YOUNG ARAB ALSO LOADED DOWN WITH BAGGAGE ENT...,A YOUNG ARAB ALSO LOADED DOWN WITH BACKAGE ENT...
3,cv-valid-dev/sample-000003.mp3,i thought that everything i owned would be des...,3,0,NaN,NaN,NaN,NaN,I FELT THAT EVERYTHING I OWNED WOULD BE DESTROYED,I THOUGHT THAT EVERYTHING I OWNED WOULD BE DES...
4,cv-valid-dev/sample-000004.mp3,he moved about invisible but everyone could he...,1,0,fourties,female,england,NaN,HE MOVED ABOUT INVISIBLE BUT EVERY ONE COULD H...,HE MOVED ABOUT INVISIBLE BUT EVERYONE COULD HE...


In [26]:
# Read the list of detected filenames
with open('/content/detected.txt', 'r') as file:
    detected_filenames = file.read().splitlines()

In [27]:
print(detected_filenames)

['cv-valid-dev/sample-000000.mp3', 'cv-valid-dev/sample-000003.mp3', 'cv-valid-dev/sample-000089.mp3', 'cv-valid-dev/sample-000508.mp3', 'cv-valid-dev/sample-000674.mp3', 'cv-valid-dev/sample-001093.mp3', 'cv-valid-dev/sample-001101.mp3', 'cv-valid-dev/sample-001243.mp3', 'cv-valid-dev/sample-001501.mp3', 'cv-valid-dev/sample-001933.mp3', 'cv-valid-dev/sample-002405.mp3', 'cv-valid-dev/sample-002453.mp3', 'cv-valid-dev/sample-003065.mp3', 'cv-valid-dev/sample-003219.mp3', 'cv-valid-dev/sample-003662.mp3', 'cv-valid-dev/sample-003808.mp3']


In [76]:
hotwords = ["destroy", "be careful", "stranger"]

# Prepare sentences for embedding from the dataset, ensuring each sentence is capitalized
# as the model's encoding might be case-sensitive
phrases_to_match = [['sentence to match: ', sentence.capitalize()] for sentence in df["finetuned_text"]]
hotword_phrases = [['phrase to find: ', hotword] for hotword in hotwords]

# Generate embeddings for each list of phrases
phrases_embeddings = model.encode(phrases_to_match)
hotword_embeddings = model.encode(hotword_phrases)

# Compute cosine similarity between each phrase and each hotword
similarity_matrix = cosine_similarity(phrases_embeddings, hotword_embeddings)

# Print the entire similarity matrix and the similarity scores for the first phrase
print("Similarity Matrix:\n", similarity_matrix)
print("First phrase similarities:", similarity_matrix[0])


Similarity Matrix:
 [[0.74289083 0.9018011  0.8842734 ]
 [0.7570061  0.80617607 0.81027174]
 [0.74464786 0.7799059  0.7998063 ]
 ...
 [0.76165843 0.8156958  0.7796242 ]
 [0.730847   0.75699335 0.78784376]
 [0.753673   0.7567514  0.76979035]]
First phrase similarities: [0.74289083 0.9018011  0.8842734 ]


In [77]:
min_scores = {}

# Calculate minimum similarity scores where the actual text contains the hotword
for idx, hotword in enumerate(hotwords):
    # Filter rows where 'finetuned_text' contains the hotword
    filtered_rows = df[df['finetuned_text'].str.contains(hotword, case=False, na=False)]

    # Gather similarity scores for these filtered rows
    similarity_scores = [similarity_matrix[row_idx][idx] for row_idx in filtered_rows.index]

    # Store the minimum score for this hotword in the dictionary
    min_scores[hotword] = min(similarity_scores) if similarity_scores else float('inf')

print("Minimum Similarity Scores:", min_scores)

Minimum Similarity Scores: {'destroy': 0.86071455, 'be careful': 0.8972255, 'stranger': 0.8272493}


In [78]:
# Determine which phrases match any hotword above its minimum similarity score
hotword_matches = []

for similarities in similarity_matrix:
    # Check each phrase's similarity to each hotword against the minimum scores
    match_found = any(similarities[idx] >= min_scores[hotwords[idx]] for idx in range(len(hotwords)))
    hotword_matches.append(match_found)

# Assign the match results back to the DataFrame
df["similarity"] = hotword_matches

In [79]:
df

,filename,text,up_votes,down_votes,age,gender,accent,duration,generated_text,finetuned_text,hotword_match
0,cv-valid-dev/sample-000000.mp3,be careful with your prognostications said the...,1,0,NaN,NaN,NaN,NaN,BE CAREFUL WITH YOUR PROGNOSTICATIONS SAID THE...,BE CAREFUL WITH YOUR PROCNOSTICATIONS SAID THE...,True
1,cv-valid-dev/sample-000001.mp3,then why should they be surprised when they se...,2,0,NaN,NaN,NaN,NaN,THEN WHY SHOULD THEY BE SURPRISED WHEN THEY SE...,THEN WHY SHOULD THEY BE SURPRISED WITH THE SEAONE,False
2,cv-valid-dev/sample-000002.mp3,a young arab also loaded down with baggage ent...,2,0,NaN,NaN,NaN,NaN,A YOUNG ARAB ALSO LOADED DOWN WITH BAGGAGE ENT...,A YOUNG ARAB ALSO LOADED DOWN WITH BACKAGE ENT...,False
3,cv-valid-dev/sample-000003.mp3,i thought that everything i owned would be des...,3,0,NaN,NaN,NaN,NaN,I FELT THAT EVERYTHING I OWNED WOULD BE DESTROYED,I THOUGHT THAT EVERYTHING I OWNED WOULD BE DES...,True
4,cv-valid-dev/sample-000004.mp3,he moved about invisible but everyone could he...,1,0,fourties,female,england,NaN,HE MOVED ABOUT INVISIBLE BUT EVERY ONE COULD H...,HE MOVED ABOUT INVISIBLE BUT EVERYONE COULD HE...,False
...,...,...,...,...,...,...,...,...,...,...,...
4071,cv-valid-dev/sample-004071.mp3,but they could never have taught him arabic,2,1,NaN,NaN,NaN,NaN,BUT THEY COULD NEVER HAVE TAUGHT HIM ARABIC,BUT THEY COULD NEVER HAVE TAUGHT HIM ARABIC,False
4072,cv-valid-dev/sample-004072.mp3,he decided to concentrate on more practical ma...,1,0,NaN,NaN,NaN,NaN,HE DECIDED TO CONCENTRATE ON MORE PRACTICAL MA...,HE DECIDED TO CONCENTRATE ON MORE PRACTICAL MA...,False
4073,cv-valid-dev/sample-004073.mp3,that's what i'm not supposed to say,2,0,thirties,male,us,NaN,THAT'S WHAT I'M NOT SUPPOSED TO SAY,THAT'S WHAT I'M NOT SUPPOSED TO SAY,False
4074,cv-valid-dev/sample-004074.mp3,just handling them made him feel better,3,0,NaN,NaN,NaN,NaN,JUST HERE THE WINPLE MADE HIM FEEL BETTER,JUST HERE DOING THIG MADE HIM FEEL BETTER,False


In [80]:
matched_df = df[df['similarity'] == True]

matched_df['finetuned_text']


0       BE CAREFUL WITH YOUR PROCNOSTICATIONS SAID THE...
3       I THOUGHT THAT EVERYTHING I OWNED WOULD BE DES...
89          THE STRANGER SEEMED SATISFIED WITH THE ANSWER
202                 HOW STRANGE AFRICA IS THOUGHT THE BOY
231                  HE FELT UNEASY AT THE MAN'S PRESENCE
                              ...                        
3808         I HAD TO TEST YOUR COURAGE THE STRANGER SAID
3843    I HAVE TO FIND T HE MEN WHO KNOWS THAT UNIVERS...
3909    THIS WAS THE STRANGEST OF ALL THINGS THAT EVER...
3921                      I'DN'T KNOW THAT VOICE ANYWHERE
4068    HE WAS THINKING ABOUT OMENS AND SOMEONE HAD AP...
Name: finetuned_text, Length: 68, dtype: object

In [81]:
df.to_csv('cv-valid-dev-results.csv')